In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nsepy import get_history

# Function to check if a stock has increased by 10%
def has_increased_by_10(percent_change):
    return percent_change >= 10.0

# Get the list of all available stock symbols in NSE
all_stocks = pd.read_csv('https://www1.nseindia.com/content/indices/ind_nifty500list.csv')
stocks = all_stocks['Symbol'].tolist()

# Logistic Regression Classifier
logistic_regression = LogisticRegression()

predictions = []

for stock in stocks:
    try:
        # Fetch the historical stock data using nsepy for 2023
        data = get_history(symbol=stock, index='NIFTY 50', start=pd.to_datetime('2023-06-20'), end=pd.to_datetime('2023-06-23'), index_as_date=True, interval='1d')

        # Shift the Close prices by one day to get the next day's price
        data['NextDayClose'] = data['Close'].shift(-1)

        # Calculate the percent change from current day's Close to next day's Close
        data['PercentChange'] = (data['NextDayClose'] - data['Close']) / data['Close'] * 100.0

        # Remove the last row with NaN value
        data = data[:-1]

        # Select the relevant features for prediction (e.g., 'Open', 'High', 'Low', 'Volume')
        features = data[['Open', 'High', 'Low', 'Volume']]
        target = data['PercentChange']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # Train the logistic regression classifier
        logistic_regression.fit(X_train, y_train)

        # Make predictions on the test set
        predictions.extend([(data.index[i].date(), stock, X_test.iloc[i], y_test.iloc[i]) for i in range(len(X_test)) if has_increased_by_10(y_test.iloc[i])])

    except:
        print(f"Error occurred for stock: {stock}")

# Sort the predictions based on percent change in descending order
predictions.sort(key=lambda x: x[3], reverse=True)

# Print the top 10 predictions
for prediction in predictions[:10]:
    date, stock, features, percent_change = prediction
    print(f"Date: {date}, Stock: {stock}, Features: {features}, Percent Change: {percent_change}")
